### We used Geneformer to inform our manual celltype annotation of the replicate data, and tell us which cells were Epithelial and Stromal classes for axes classification

In [1]:
import sys
sys.path.append('/home/amonell/Geneformer')
from geneformer import TranscriptomeTokenizer
import numpy as np
import os

/home/amonell/mambaforge/envs/geneformer/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/home/amonell/mambaforge/envs/geneformer/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:


### Put the path to the reference data

In [2]:
import scanpy as sc
ad=sc.read('/mnt/sata1/Analysis_Alex/timecourse_final/analysis/cleaned/final_celltyped_and_axes.h5ad')

/home/amonell/.local/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/amonell/.local/lib/python3.10/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
import pandas as pd
pd.DataFrame(ad.var.index.tolist()).to_csv(r'/mnt/sata1/Analysis_Alex/Geneformer/adcsv.csv', index=None)

### Converting gene names to human homolog ensembl ids

In [4]:
ensembl = pd.read_csv('/mnt/sata1/Analysis_Alex/Geneformer/gProfiler_hsapiens_9-18-2023_2-31-49 PM.csv').drop_duplicates('initial_alias')

In [7]:
ad.var['ensembl_id'] = [i for i in ensembl['converted_alias'] if not pd.isna(i)][:len(ad.var.index)]
ad.var.index= ad.var.ensembl_id.tolist()
ad.obs['n_counts'] = np.array(np.sum(ad.X, axis=1)).flatten()
ad.obs['organ_major'] = "SI"

In [1]:
os.mkdir('/mnt/sata1/Analysis_Alex/Geneformer/loom_xenium')

In [12]:
ad.write_loom('/mnt/sata1/Analysis_Alex/Geneformer/loom_xenium/ad.loom')

The loom file will lack these fields:
{'X_scVI', 'X_mde', 'X_spatial'}
Use write_obsm_varm=True to export multi-dimensional annotations


In [13]:
ad.obs['input_ids'] = [i for i in range(len(ad.obs.index.tolist()))]
d = {}
for i in ad.var.index.tolist():
    d[i] = True
import pickle
with open("/mnt/sata1/Analysis_Alex/Geneformer/ids.pkl", 'wb') as w:
    pickle.dump(d, w)

In [14]:
tk = TranscriptomeTokenizer({"Subtype": "cell_type", "organ_major": "organ_major"}, nproc=4)
tk.tokenize_data("/mnt/sata1/Analysis_Alex/Geneformer/loom_xenium", "/mnt/sata1/Analysis_Alex/Geneformer/loom_xenium/tokenized", "train_xenium")

Tokenizing /mnt/sata1/Analysis_Alex/Geneformer/loom_xenium/ad.loom
/mnt/sata1/Analysis_Alex/Geneformer/loom_xenium/ad.loom has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


Saving the dataset (1/1 shards): 100%|██████████| 1763585/1763585 [00:00<00:00, 3193315.34 examples/s]


### Tokenize datasets to annotate

In [ ]:
import glob
ensembl = pd.read_csv('/mnt/sata1/Analysis_Alex/Geneformer/gProfiler_hsapiens_9-18-2023_2-31-49 PM.csv').drop_duplicates('initial_alias')

for filename in glob.glob('/mnt/sata1/Analysis_Alex/timecourse_replicates/day*'):
    outname = os.path.basename(filename)
    try:
        os.mkdir('/mnt/sata1/Analysis_Alex/Geneformer/loom_'+outname)
        ad=sc.read(os.path.join('/mnt/sata1/Analysis_Alex/timecourse_replicates', outname, 'adatas/06_reference_mapped.h5ad'))
        ad.var['ensembl_id'] = [i for i in ensembl['converted_alias'] if not pd.isna(i)][:len(ad.var.index)]
        ad.var.index= ad.var.ensembl_id.tolist()
        ad.obs['n_counts'] = np.array(np.sum(ad.X, axis=1)).flatten()
        ad.obs['organ_major'] = "SI"
        ad.obs['Subtype'] = 0
        ad.write_loom('/mnt/sata1/Analysis_Alex/Geneformer/loom_'+outname+'/'+outname+'.loom')

        tk = TranscriptomeTokenizer({"Subtype": "cell_type", "organ_major": "organ_major"}, nproc=4)
        tk.tokenize_data("/mnt/sata1/Analysis_Alex/Geneformer/loom_"+outname, "/mnt/sata1/Analysis_Alex/Geneformer/loom_"+outname+"/tokenized", "train_"+outname)
    except:
        print(outname)
